### Click on link for data I used
https://www.kaggle.com/therohk/india-headlines-news-dataset



In [2]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
import pandas as pd
np.random.seed(10)

from nltk import word_tokenize, pos_tag

In [3]:
import pandas as pd


data = pd.read_csv('india-news-headlines.csv', error_bad_lines=False);

pd.set_option('display.max_colwidth', 100)

data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text
data_text.head(20)
# data

C:\Users\Sam\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,headline_text,index
0,win over cena satisfying but defeating undertaker bigger roman reigns,0
1,Raju Chacha,1
2,Status quo will not be disturbed at Ayodhya; says Vajpayee,2
3,Fissures in Hurriyat over Pak visit,3
4,America's unwanted heading for India?,4
5,For bigwigs; it is destination Goa,5
6,Extra buses to clear tourist traffic,6
7,Dilute the power of transfers; says Riberio,7
8,Focus shifts to teaching of Hindi,8
9,IT will become compulsory in schools,9


In [4]:
# 811010	australia needs to rediscover belief: warner
# 788894	'shell shocked' oklahoma residents vow to rebuild
# 1130409	art of the flip: why bankers have turned snitch

# data_text1 = documents.sample(frac=1)
# data_text1.head(25)

### Deal with all type of useful words

#### Lemmatisation, tokenize, remove STOPWORDS

gensim.corpora.Dictionary(text_document) ==> Dictionary encapsulates the mapping between normalized words and their integer ids.

doc2bow(document, allow_update=False, return_missing=False)
Convert document into the bag-of-words (BoW) format = list of (token_id, token_count) tuples.


Filter out tokens:
    * less than 15 documents (absolute number) or
    * more than 0.5 documents
    * after the above two steps, keep only the first 100000 most frequent tokens

bow_corpus ==> created a dictionary reporting how many
words and how many times those words appear

LdaMulticore ==> Online Latent Dirichlet Allocation (LDA) in Python, using all CPU cores to parallelize and speed up model training.

The parallelization uses multiprocessing;


Topic: 0 : Meeting and women <br/>
Topic: 1 : New year <br/>
Topic: 2 : Politics <br/>
Topic: 3 : word related to  <br/>
Topic: 4 :  <br/>
Topic: 5 :  <br/>
Topic: 6 : khan + car + shoot, bollywood <br/>
Topic: 7 : student protest <br/>
Topic: 8 : news related to houses (rajya & lokh shabha) <br/>
Topic: 9 :  <br/>


### Speed ===> LdaMulticore (workers=3) > LdaMulticore (workers=2) > LdaMulticore (workers=1) > LdaModel

Topic: 0 :  <br/>
Topic: 1 :  <br/>
Topic: 2 :  <br/>
Topic: 3 :   <br/>
Topic: 4 :  <br/>

The above two lines took a lot more time to run but toic name not making any sense!!

Topic not making that much sense so looking noun and adjective only

## Deal with noun and adjectives

Here only 'NN' and 'JJ'

In [5]:
documents.head(15)

,headline_text,index
0,win over cena satisfying but defeating undertaker bigger roman reigns,0
1,Raju Chacha,1
2,Status quo will not be disturbed at Ayodhya; says Vajpayee,2
3,Fissures in Hurriyat over Pak visit,3
4,America's unwanted heading for India?,4
5,For bigwigs; it is destination Goa,5
6,Extra buses to clear tourist traffic,6
7,Dilute the power of transfers; says Riberio,7
8,Focus shifts to teaching of Hindi,8
9,IT will become compulsory in schools,9


In [6]:
import re
import string

stemmer = SnowballStemmer('english')

def lemmatize_stemming2(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))


def preprocess2(text):
#     text = re.sub('[%s]' % re.escape(string.punctuation), '', text)         # remove punctuation
#     text = re.sub('\w*\d\w*', '', text)                                     # \d ==> [0-9] , \w ==> a-z A-Z 0-9, \W => non alphanumerical value #remove words containing numbers
                                                                             # This take long computation time due to 20 Lakh headlines we deal this in different way
    result2 = []
    a = []
        
    for token in gensim.utils.simple_preprocess(text, min_len=3):
        a.append(token)
    
    noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    
    for (word, pos) in pos_tag(a):
        if noun_adj(pos):
            if word not in gensim.parsing.preprocessing.STOPWORDS:   # stop words
                result2.append(lemmatize_stemming2(word))
    a = []
    return result2

In [7]:
documents2 = documents['headline_text'].map(preprocess2)  # replace headline value with output of preprocess function 
                                                        # documents.headline_text.apply(a_preprocess2) takes a lot time
# documents2['index'] = data_text.index
documents2.head(15)

0     [win, cena, undertak, bigger, roman, reign]
1                                  [raju, chacha]
2                 [status, quo, ayodhya, vajpaye]
3                            [fissur, pak, visit]
4                           [unwant, head, india]
5                           [bigwig, destin, goa]
6           [extra, bus, clear, tourist, traffic]
7                      [power, transfer, riberio]
8                           [focus, shift, hindi]
9                            [compulsori, school]
10              [stop, freedom, fighter, pension]
11                [gilani, claim, passport, year]
12                             [parivar, dismiss]
13                    [pak, exchang, list, plant]
14                                         [govt]
Name: headline_text, dtype: object

#### EDA (analize starting 1 Lakh headlines)

In [25]:
document3 = pd.DataFrame(documents2).transpose()
# documents3['index'] = data_text.index
# document3.head(15)

In [26]:
document3.head(15)

,0,1,2,3,4,5,6,7,8,9,...,2969912,2969913,2969914,2969915,2969916,2969917,2969918,2969919,2969920,2969921
headline_text,"[win, cena, undertak, bigger, roman, reign]","[raju, chacha]","[status, quo, ayodhya, vajpaye]","[fissur, pak, visit]","[unwant, head, india]","[bigwig, destin, goa]","[extra, bus, clear, tourist, traffic]","[power, transfer, riberio]","[focus, shift, hindi]","[compulsori, school]",...,"[curious, case, vowel]","[matti, shanahan, pentagon, helm, critic, time]","[republican, trump, vow, syria]","[tattoo, athlet, avatar, problem]","[donald, trump, promis, wall]","[guest, touch, venu]","[guatemala, villag, mourn, die, custodi]","[veteran, support, donald, trump, strong, poll]","[aid, signal, trump, wall]","[throng, pack, time, mild, new, year]"


In [70]:
tuple(document3[0])

(['win', 'cena', 'undertak', 'bigger', 'roman', 'reign'],)

In [71]:
# easy methon to get rid of " TypeError: unhashable type: 'list' "

# document3.columns
# print(document3[0].dtype)

# from collections import Counter
# a = tuple(document3[0])[0] 
# a = tuple(document3[0])[0] + tuple(document3[1])[0]
# print(a)

# print(Counter(a).most_common())

In [77]:
from collections import Counter
List = tuple(document3[0])[0]
for i in range(100000):
    List = List + tuple(document3[i+1])[0]

print(Counter(List).most_common()) 

[('govt', 2809), ('india', 1974), ('new', 1895), ('polic', 1765), ('bjp', 1592), ('citi', 1585), ('case', 1409), ('meet', 1230), ('state', 1211), ('power', 1170), ('poll', 1169), ('cong', 1111), ('day', 1094), ('plan', 1069), ('pak', 988), ('water', 909), ('minist', 897), ('indian', 887), ('student', 875), ('centr', 861), ('chief', 809), ('time', 789), ('court', 786), ('life', 747), ('road', 736), ('bank', 730), ('terror', 730), ('leader', 727), ('school', 724), ('panel', 714), ('offic', 709), ('man', 699), ('today', 688), ('year', 681), ('offici', 677), ('issu', 673), ('secur', 672), ('hous', 665), ('order', 664), ('parti', 633), ('death', 612), ('project', 603), ('gujarat', 598), ('women', 588), ('probe', 585), ('strike', 584), ('law', 581), ('attack', 574), ('congress', 572), ('report', 537), ('delhi', 530), ('tax', 522), ('murder', 520), ('talk', 519), ('film', 518), ('fund', 512), ('train', 487), ('servic', 486), ('high', 482), ('work', 480), ('world', 473), ('plea', 464), ('bus',

In [10]:
for i in np.arange(14):
    k = documents2[i]
    print(pos_tag(k))

[('win', 'VB'), ('cena', 'NN'), ('undertak', 'JJ'), ('bigger', 'JJR'), ('roman', 'NN'), ('reign', 'NN')]
[('raju', 'NN'), ('chacha', 'NN')]
[('status', 'NN'), ('quo', 'NN'), ('ayodhya', 'NN'), ('vajpaye', 'NN')]
[('fissur', 'JJ'), ('pak', 'NN'), ('visit', 'NN')]
[('unwant', 'JJ'), ('head', 'NN'), ('india', 'NN')]
[('bigwig', 'NN'), ('destin', 'NN'), ('goa', 'NN')]
[('extra', 'JJ'), ('bus', 'NN'), ('clear', 'JJ'), ('tourist', 'NN'), ('traffic', 'NN')]
[('power', 'NN'), ('transfer', 'NN'), ('riberio', 'NN')]
[('focus', 'NN'), ('shift', 'NN'), ('hindi', 'NN')]
[('compulsori', 'JJ'), ('school', 'NN')]
[('stop', 'NN'), ('freedom', 'NN'), ('fighter', 'NN'), ('pension', 'NN')]
[('gilani', 'NN'), ('claim', 'NN'), ('passport', 'NN'), ('year', 'NN')]
[('parivar', 'NN'), ('dismiss', 'NN')]
[('pak', 'NN'), ('exchang', 'NN'), ('list', 'NN'), ('plant', 'NN')]


In [11]:
dictionary2 = gensim.corpora.Dictionary(documents2)

In [12]:
dictionary2.filter_extremes(no_below=15, no_above=0.6, keep_n=100000)  # Filter out tokens in the dictionary by their frequency.

In [13]:
bow_corpus2 = [dictionary2.doc2bow(doc) for doc in documents2]

In [14]:
bow_corpus2[190]

[(310, 1)]

In [15]:
print('id', bow_corpus2[190][0][0], 'word ("', dictionary2[bow_corpus2[190][0][0]], '") appered', bow_corpus2[190][0][1], 'times')

id 310 word (" muse ") appered 1 times


In [16]:
lda_model4 = gensim.models.LdaMulticore(bow_corpus2, num_topics=12, id2word=dictionary2, passes=2, workers=2)

In [17]:
lda_model4.print_topics()

[(0,
  '0.052*"india" + 0.046*"bjp" + 0.017*"kid" + 0.017*"leader" + 0.016*"telangana" + 0.016*"share" + 0.014*"servic" + 0.014*"strike" + 0.013*"goa" + 0.012*"maharashtra"'),
 (1,
  '0.058*"road" + 0.024*"meet" + 0.023*"bus" + 0.020*"traffic" + 0.020*"indian" + 0.020*"open" + 0.020*"farmer" + 0.020*"parti" + 0.016*"assembl" + 0.015*"youth"'),
 (2,
  '0.046*"woman" + 0.030*"women" + 0.025*"month" + 0.017*"son" + 0.016*"kapoor" + 0.016*"minor" + 0.015*"gold" + 0.015*"metro" + 0.014*"rule" + 0.012*"chennai"'),
 (3,
  '0.048*"school" + 0.043*"congress" + 0.041*"state" + 0.031*"time" + 0.027*"film" + 0.023*"modi" + 0.022*"hospit" + 0.021*"rajasthan" + 0.015*"drive" + 0.015*"fund"'),
 (4,
  '0.040*"govern" + 0.035*"water" + 0.027*"updat" + 0.027*"crore" + 0.026*"minist" + 0.017*"dead" + 0.015*"attack" + 0.014*"free" + 0.012*"unit" + 0.012*"pollut"'),
 (5,
  '0.066*"man" + 0.033*"girl" + 0.032*"elect" + 0.031*"death" + 0.023*"centr" + 0.020*"famili" + 0.020*"offic" + 0.020*"offici" + 0.017*"

In [18]:
lda_model5 = gensim.models.LdaMulticore(bow_corpus2, num_topics=8, id2word=dictionary2, passes=2, workers=2)

In [19]:
lda_model5.print_topics()

[(0,
  '0.041*"man" + 0.027*"woman" + 0.016*"updat" + 0.015*"crore" + 0.015*"film" + 0.012*"delhi" + 0.010*"assembl" + 0.010*"youth" + 0.010*"son" + 0.010*"wife"'),
 (1,
  '0.024*"bjp" + 0.021*"congress" + 0.018*"elect" + 0.016*"high" + 0.014*"meet" + 0.013*"singh" + 0.012*"parti" + 0.012*"farmer" + 0.011*"modi" + 0.011*"minist"'),
 (2,
  '0.054*"year" + 0.029*"old" + 0.023*"school" + 0.019*"lakh" + 0.018*"water" + 0.016*"plan" + 0.010*"resid" + 0.010*"home" + 0.010*"villag" + 0.009*"dead"'),
 (3,
  '0.031*"case" + 0.029*"student" + 0.017*"court" + 0.012*"colleg" + 0.010*"order" + 0.010*"report" + 0.010*"probe" + 0.009*"post" + 0.009*"rape" + 0.008*"gujarat"'),
 (4,
  '0.024*"govern" + 0.016*"today" + 0.012*"hospit" + 0.012*"peopl" + 0.011*"pradesh" + 0.009*"vehicl" + 0.009*"rule" + 0.008*"seat" + 0.008*"driver" + 0.007*"doctor"'),
 (5,
  '0.032*"cop" + 0.025*"polic" + 0.016*"women" + 0.016*"offic" + 0.014*"month" + 0.013*"bank" + 0.011*"rajasthan" + 0.010*"illeg" + 0.010*"station" + 0

### This make sense to me!

In [20]:
lda_model3 = gensim.models.LdaMulticore(bow_corpus2, num_topics=10, id2word=dictionary2, passes=2, workers=2)

In [21]:
lda_model3.print_topics()

[(0,
  '0.039*"case" + 0.028*"court" + 0.020*"high" + 0.017*"crore" + 0.016*"murder" + 0.016*"centr" + 0.015*"bank" + 0.014*"project" + 0.013*"order" + 0.013*"rajasthan"'),
 (1,
  '0.026*"citi" + 0.024*"elect" + 0.023*"india" + 0.019*"updat" + 0.017*"singh" + 0.017*"home" + 0.016*"world" + 0.016*"open" + 0.014*"peopl" + 0.013*"air"'),
 (2,
  '0.049*"road" + 0.022*"mumbai" + 0.017*"car" + 0.013*"garbag" + 0.012*"way" + 0.012*"rain" + 0.011*"resid" + 0.011*"tamil" + 0.010*"fund" + 0.010*"jail"'),
 (3,
  '0.051*"day" + 0.027*"water" + 0.019*"month" + 0.017*"power" + 0.014*"villag" + 0.014*"dead" + 0.013*"kid" + 0.013*"kerala" + 0.012*"gujarat" + 0.012*"accid"'),
 (4,
  '0.020*"minist" + 0.018*"univers" + 0.018*"traffic" + 0.018*"chief" + 0.015*"report" + 0.014*"pradesh" + 0.013*"new" + 0.013*"rape" + 0.013*"bengaluru" + 0.011*"corpor"'),
 (5,
  '0.063*"year" + 0.044*"man" + 0.035*"student" + 0.034*"old" + 0.032*"new" + 0.029*"woman" + 0.022*"girl" + 0.016*"death" + 0.014*"farmer" + 0.012*

Topic: 0 : Politics   <br/>
Topic: 1 : Roads related (traffic, bus, die)   <br/>
Topic: 2 : women   <br/>
Topic: 3 : mix ('school', 'congress', 'state')   <br/>
Topic: 4 : Water <br/>
Topic: 5 : Rajasthan death <br/>
Topic: 6 : man  <br/>
Topic: 7 : rape related  <br/>
Topic: 8 : khan(may be salman related), case & cop   <br/>
Topic: 9 : time (today, day, week) <br/>